In [23]:
import requests
import re
from bs4 import BeautifulSoup

In [1]:
A = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12']
B = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12']
C = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12']

MERGED = [item for sublist in zip(A, B, C) for item in sublist]
# Define a generator function
def generate_merged_list():
    for element in MERGED:
        yield element

In [1]:
site_variables = {
    'thesun': {
        'main' : 'https://www.thesun.co.uk',
        'secondary' : [
            'https://www.thesun.co.uk/news/',
            'https://www.thesun.co.uk/news/worldnews/',
            'https://www.thesun.co.uk/news/uknews/',
            'https://www.thesun.co.uk/news/opinion/'
        ],
        'sport' : 
            [
                'https://www.thesun.co.uk/sport/',
                'https://www.thesun.co.uk/sport/football/',
                'https://www.thesun.co.uk/sport/boxing/',
                'https://www.thesun.co.uk/sport/mma/',
                'https://www.thesun.co.uk/sport/horseracing/',
                'https://www.thesun.co.uk/sport/golf/',
                'https://www.thesun.co.uk/sport/wwe/',
                'https://www.thesun.co.uk/sport/tennis/',
                'https://www.thesun.co.uk/sport/motorsport/',
                'https://www.thesun.co.uk/sport/rugbyunion/',
                'https://www.thesun.co.uk/sport/cricket/',                
                
                
            ],
        'technology' : [
            'https://www.thesun.co.uk/tech/',
            'https://www.thesun.co.uk/tech/news-tech/',
            'https://www.thesun.co.uk/tech/phones-gadgets/',
            'https://www.thesun.co.uk/tech/gaming/'
            ],
        'health' : [
            'https://www.thesun.co.uk/health/',
            'https://www.thesun.co.uk/news/health-news/',
            'https://www.thesun.co.uk/health/news-health/',
            'https://www.thesun.co.uk/health/wellbeing/',
            'https://www.thesun.co.uk/health/mens-health-and-fitness/',
            'https://www.thesun.co.uk/health/womens-health/',
            'https://www.thesun.co.uk/health/diet-fitness/',
            'https://www.thesun.co.uk/health/mental-health/',
            'https://www.thesun.co.uk/topic/fabulous-health/'          
        ],
        'science' : 'https://www.thesun.co.uk/news/science/',
        'politics' : 'https://www.thesun.co.uk/news/politics/',
        'page_suffix' : 'page/{number}/', 
        'source_site' : 'The Sun UK',
        'paragraph_tag' : 'p',
        'title_tag' : 'h1',
        'exclude' : ['content__meta', 'content__meta--byline', 'breaking-news__headline'] 
    },
    'cbsnews' : {
        'main' : 'https://www.cbsnews.com',
        'secondary' : [
            'https://www.cbsnews.com/latest/world/',
            'https://www.cbsnews.com/latest/us/',
            'https://www.cbsnews.com/latest/entertainment/',
            'https://www.cbsnews.com/crime/'
        ],
        'sport' : [
            'https://www.cbssports.com/',
            'https://www.cbssports.com/nfl/',
            'https://www.cbssports.com/nba/',
            'https://www.cbssports.com/nhl/',
            'https://www.cbssports.com/mlb/',
            'https://www.cbssports.com/college-football/',
            'https://www.cbssports.com/college-basketball/',
            'https://www.cbssports.com/soccer/',
            'https://www.cbssports.com/golf/',
            'https://www.cbssports.com/tennis/',
            
        ],
        'technology' : 'https://www.cbsnews.com/latest/technology',
        'health' : 'https://www.cbsnews.com/latest/healthwatch/',
        'science' : 'https://www.cbsnews.com/latest/science/',
        'politics' : 'https://www.cbsnews.com/latest/politics/',
        'source_site' : 'CBS News',
        'page_suffix' : {
            'all' : '{number}',
            'sport' : 'https://www.cbssports.com/{category}/{number}/',
        },
        'paragraph_tag' : 'p',
        'title_tag' : 'h1',
        'exclude' : ['content__meta', 'content__meta--byline', 'breaking-news__headline']
    },
    'abcnews' : {
        'main' : 'https://abcnews.go.com',
        'secondary' : [
                'https://abcnews.go.com/US',
                'https://abcnews.go.com/International',
                'https://abcnews.go.com/Alerts/Weather',
                'https://abcnews.go.com/Entertainment'
                
            ],
        'sport' : 'https://abcnews.go.com/Sports',
        'technology' : 'https://abcnews.go.com/Technology',
        'health' : 'https://abcnews.go.com/Health',
        'politics' : 'https://abcnews.go.com/Politics',
        'science' : None,
        'source_site' : 'ABC News',
        'paragraph_tag' : 'p',
        'title_tag' : 'h1',
        'exclude' : ['Article__Header__Branding', 'PinnedPostsContainer']
    },
    'nbcuniversal' : {
        'source_site' : 'NBC Universal',
        'paragraph_tag' : 'p',
        'title_tag' : 'h1',
        'exclude' : ['block-socialblock', 'social-btns', 'social-links-margin', 'block-copyrightblock', 'copyright print-no']
    },
}



SyntaxError: invalid syntax (1166294463.py, line 8)

In [3]:
special_chars_trans = str.maketrans({
    '\n': ' ', 
    '\xa0': ' ',
    '\t': ' ',
    "\'": "'"
})

def scrape_news(url:str, paragraph_tag:str, title_tag:str, subtitle_tag:str = None, exclude:list[str] = []) -> dict[str, str]:
    response = requests.get(url)
    if  response.status_code != 200:
        #  Failed to retrieve content
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find(title_tag).text
    subtitle = soup.find(subtitle_tag).text if subtitle_tag else None
     
    content = ''
    content_list = []
    paragraphs = soup.find_all(paragraph_tag)
    for paragraph in paragraphs:
        wrapper_elements_class = paragraph.find_parent(class_=exclude)
        wrapper_elements_id = paragraph.find_parent(id=exclude)
        
        
        element_class = paragraph.get('class', None)
        common_class = {}
        if element_class and exclude:
            common_class = set(element_class).intersection(exclude)
            
        element_id = paragraph.get('id', None)
        common_id = {}
        if element_id and exclude:
            common_id = set(element_id).intersection(exclude)
            
        if wrapper_elements_class or wrapper_elements_id or common_class or common_id:
            continue

        content_list.append(paragraph.text.strip())
        content = ' '.join(content_list)

    
    result = {
        'title': title.translate(special_chars_trans).strip() if title else None,
        'subtitle': subtitle.translate(special_chars_trans).strip() if subtitle else None,
        'content': content.translate(special_chars_trans).strip() if content else None,
    }
 
    return result


In [4]:
def scraper(url:str):
    site_dict = next((site_dict for key,site_dict in site_variables.items() if url.startswith(f'https://www.{key}')), None)
    if not site_dict:
        # some pop up
        return None
    result_dict = scrape_news(url, site_dict['paragraph_tag'], site_dict['title_tag'], exclude=site_dict['exclude'])
    result_dict['source_site'] = site_dict['source_site']
    return result_dict

In [5]:
res = scraper('https://www.nbcuniversal.com/article/nbcuniversal-puppy-training-program-begins-30-rock')


In [6]:
res

{'title': 'NBCUniversal Partners with America’s Vet Dogs: Service Dog Training at 30 Rock Begins',
 'subtitle': None,
 'content': "NBCUniversal collaborates with America’s Vet Dogs and their sister organization the Guide Dog Foundation for a unique Puppy-in-Training Program at 30 Rock. Discover how this initiative promotes accessibility and the long-standing commitment of NBCUniversal of supporting service animals. November 10, 2023 In an effort to promote accessibility and educate employees, NBCUniversal has entered into an exciting partnershp with America’s Vet Dogs and their sister organization, the Guide Dog Foundation, on the Puppy-In-Training program. This initiative provides a comforting and valuable experience for employees, and also speaks to NBCUniversal’s commitment to accessibility. For 20 years, America’s VetDogs has trained and placed guide and service dogs to provide independence, enhanced mobility, and companionship to veterans with disabilities from all eras. In 2015, 

In [20]:
# https://www.cbsnews.com/world/
# url = 'https://www.cbsnews.com/politics/'
# https://www.cbsnews.com/entertainment/ 
# https://www.cbsnews.com/us/
# https://www.cbsnews.com/healthwatch/ 
# https://www.cbsnews.com/moneywatch/ 
# https://www.cbsnews.com/cbsvillage/ 
# https://www.cbsnews.com/technology/ 
# https://www.cbsnews.com/science/ 
# url = 'https://www.cbsnews.com/crime/'
# url = 'https://www.cbsnews.com/cbs-essentials/'
# url = 'https://www.cbsnews.com'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
matching_hrefs = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('https://www.cbsnews.com/news/') or a['href'].startswith('https://www.cbsnews.com/essentials/')]
matching_hrefs = list(dict.fromkeys(matching_hrefs))
matching_hrefs



In [31]:
# https://abcnews.go.com/Politics
# https://abcnews.go.com/Entertainment
# https://abcnews.go.com/Technology
# https://abcnews.go.com/Health
# https://abcnews.go.com/Alerts/Weather
# https://abcnews.go.com/Sports
# https://abcnews.go.com/US/
# https://abcnews.go.com/VR
# https://abcnews.go.com/Lifestyle
# https://abcnews.go.com/Business
# https://abcnews.go.com/International
url = 'https://abcnews.go.com/' 

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Define the patterns

photo_pattern = re.compile(r'https://abcnews.go.com/.*/photos/.*')
video_pattern = re.compile(r'https://abcnews.go.com/.*/video/.*')

# Filter hrefs based on the patterns
matching_hrefs = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('https://abcnews.go.com/')  and not ( photo_pattern.match(a['href']) or  video_pattern.match(a['href']))]
matching_hrefs = list(dict.fromkeys(matching_hrefs))
matching_hrefs


['https://abcnews.go.com/International/live-updates?id=104837945',
 'https://abcnews.go.com/Politics/house-set-vote-johnson-plan-avert-shutdown-hell/story?id=104877426',
 'https://abcnews.go.com/US/ohio--students-bus-crash/story?id=104877429',
 'https://abcnews.go.com/GMA/Living/veterinarians-warn-rising-cases-mystery-dog-illness/story?id=104906650',
 'https://abcnews.go.com/US/georgia-prosecutors-emergency-protective-order-trump-case-video-disclosure/story?id=104882446',
 'https://abcnews.go.com/US/authorities-urgently-search-man-missing-after-falling-off/story?id=104902784',
 'https://abcnews.go.com/US/wireStory/mother-boy-6-shot-teacher-faces-sentencing-marijuana-104900298',
 'https://abcnews.go.com/US/ohio--students-bus-crash/story?id=104877429',
 'https://abcnews.go.com/US/wireStory/biden-announces-5-federal-judicial-nominees-stresses-varied-104905851',
 'https://abcnews.go.com/US/wireStory/new-jersey-lady-tammy-murphy-announces-run-us-104904211',
 'https://abcnews.go.com/Business

In [35]:
# https://www.thesun.co.uk/shopping/
# https://www.thesun.co.uk/betting/
# https://www.thesun.co.uk/web-stories/
# https://www.thesun.co.uk/puzzles/
# https://www.thesun.co.uk/dear-deidre/
# https://www.thesun.co.uk/royals/
# https://www.thesun.co.uk/tech/
# https://www.thesun.co.uk/motors/
# https://www.thesun.co.uk/health/
# https://www.thesun.co.uk/travel/
# https://www.thesun.co.uk/money/
# https://www.thesun.co.uk/tvandshowbiz/
# https://www.thesun.co.uk/tv/
# https://www.thesun.co.uk/fabulous/
# https://www.thesun.co.uk/sport/
# https://www.thesun.co.uk/news/ 
url = 'https://www.thesun.co.uk' 

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Define the patterns
thesun_pattern = re.compile(r'https://www.thesun.co.uk/.*/\d+/.*')



# photo_pattern = re.compile(r'https://abcnews.go.com/.*/photos/.*')
# video_pattern = re.compile(r'https://abcnews.go.com/.*/video/.*')

# Filter hrefs based on the patterns
matching_hrefs = [a['href'] for a in soup.find_all('a', href=True) if thesun_pattern.match(a['href'])]
matching_hrefs = list(dict.fromkeys(matching_hrefs))
matching_hrefs


['https://www.thesun.co.uk/news/politics/24749434/rishi-sunak-addresses-nation-on-rwanda/',
 'https://www.thesun.co.uk/royals/24741675/morale-prince-harrys-invictus-games-low-chiefs-sacked/',
 'https://www.thesun.co.uk/tv/24751133/rebekah-vardy-itv-celebrity-big-brother/',
 'https://www.thesun.co.uk/money/24747262/asda-shopper-shares-trick-get-terrys-chocolate-orange-75p/',
 'https://www.thesun.co.uk/tvandshowbiz/celebrities/24748309/jennifer-aniston-breaks-silence-matthew-perrys-death/',
 'https://www.thesun.co.uk/news/24743714/bbc-apologise-false-reporting-israel-hospital/',
 'https://www.thesun.co.uk/news/24746888/martin-lewis-apology-itv-money-show/',
 'https://www.thesun.co.uk/news/24744913/moment-man-has-instant-karma-after-being-tased/',
 'https://www.thesun.co.uk/news/24747835/teacher-sex-pupil-texts-love-videos/',
 'https://www.thesun.co.uk/news/24732244/terrifying-drug-sweeping-uk-seaside-towns/',
 'https://www.thesun.co.uk/news/24738134/rishi-sunak-unite-mps-braverman/',
 'h

In [36]:
# THE SUN US
# https://www.the-sun.com/news/
# https://www.the-sun.com/sport/
# https://www.the-sun.com/lifestyle/
# https://www.the-sun.com/entertainment/
# https://www.the-sun.com/money/
# https://www.the-sun.com/tech/
# https://www.the-sun.com/motors/
# https://www.the-sun.com/health/
# https://www.the-sun.com/travel/
# https://www.the-sun.com/royals/
# https://www.the-sun.com/betting/
# https://www.the-sun.com/shopping/
url = 'https://www.the-sun.com'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Define the patterns
thesunus_pattern = re.compile(r'https://www.the-sun.com/.*/\d+/.*')

matching_hrefs = [a['href'] for a in soup.find_all('a', href=True) if thesunus_pattern.match(a['href'])]
matching_hrefs = list(dict.fromkeys(matching_hrefs))
matching_hrefs

['https://www.the-sun.com/entertainment/9633944/jada-pinkett-smith-legal-action-sex-claims-will-smith/',
 'https://www.the-sun.com/entertainment/9634398/the-view-ana-navarro-looks-like-teenager-weight-loss/',
 'https://www.the-sun.com/entertainment/9635900/lisa-kudrow-mourns-matthew-perry-tragic-death/',
 'https://www.the-sun.com/entertainment/9633632/gma-tj-holmes-unveils-new-tattoo-amy-robach-marathon/',
 'https://www.the-sun.com/news/9635665/jesus-birthplace-bethlehem-officially-cancels-christmas-israel-war-gaza/',
 'https://www.the-sun.com/sport/9631909/devon-wylie-dead-nfl-kansas-city-chiefs-tennessee-titans/',
 'https://www.the-sun.com/news/9632257/tiffanie-lucas-bad-path-never-imagined-this/',
 'https://www.the-sun.com/news/9630811/chilling-new-video-tiffanie-lucas-years-before-murders/',
 'https://www.the-sun.com/news/9628364/arkansas-lottery-lawsuit-rivera-palma-quinteros-boss-exclusive/',
 'https://www.the-sun.com/entertainment/9633800/extreme-weight-loss-brandi-mallorys-deat